### Task 0 Before your go

> 1. Rename Assignment-02-###.ipynb where ### is your student ID.
> 2. The deadline of Assignment-02 is 23:59pm, 04-21-2024
> 3. In this assignment, you will use word embeddings to explore our Wikipedia dataset.

### Task 1 Train word embeddings using SGNS 
> Use our enwiki-train.json as training data. You can use the [Gensim tool](https://radimrehurek.com/gensim/models/word2vec.html). But it is recommended to implement by yourself. You should explain how hyper-parameters such as dimensionality of embeddings, window size, the parameter of negative sampling strategy, and initial learning rate have been chosen.

In [26]:
# import some necessary libraries
from typing import List, Dict, Callable
from collections import defaultdict
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import utils
import json
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq

In [27]:
# load the train and test data from the json file

# NOTE: The function is inherited from my solution of assignment 1
def load_json(file_path: str) -> List:
    """
    Fetch the data from `.json` file and concat them into a list.

    Input:
    - file_path: The relative file path of the `.json` file

    Returns:
    - join_data_list: A list containing the data, with the format of [{'title':<>, 'label':<>, 'text':<>}, {}, ...]
    """
    join_data_list = []
    with open(file_path, "r") as json_file:
        for line in json_file:
            line = line.strip()
            # guaranteen the line is not empty
            if line: 
                join_data_list.append(json.loads(line))
    return join_data_list

train_file_path, test_file_path = "enwiki-train.json", "enwiki-test.json"
train_data_list, test_data_list = map(load_json, [train_file_path, test_file_path])

class Corpus:
    def __iter__(self):
        for line in train_data_list:
            yield utils.simple_preprocess(line["text"])


In [28]:
class MyWord2Vec:
    def __init__(self, text: List[dict], dimensionality: int=100, window_size: int=5, negative_samples: int=5, lr: float=0.001) -> None:
        """
        Args:
        - text: The training data
        - dimensionality: The dimension of the word embeddings
        - window_size: The size of the context window
        - negative_samples: The number of negative samples
        - lr: Learning rate of the algorithm
        """
        self.dim = dimensionality
        self.window = window_size
        self.neg = negative_samples
        self.lr = lr
        self.__vocab = set()
        self.__word_frq = defaultdict(int)
        self.__word2idx = {}
        self.__idx2word = {}
        self.__embedding = None
        self.__context_words = []
        self.__context_targets = []
        self.__build(text)
        

    def __preprocess(self, text: List[str]) -> None:
        """
        Calculate the vocabulary and the frequency of each word in the training data, while maintaining the (idx, word) map.
        """
        for sample in text:
            words = sample["text"].split()
            for word in words:
                self.__vocab.add(word)
                self.__word_frq[word] += 1
        for idx, word in enumerate(self.__vocab):
            self.__word2idx[word] = idx
            self.__idx2word[idx] = word

    def __generate_training_data(self, text: List[str]) -> None:
        """
        Generate training data from each window and save them in `self.__context_words` and `self.__context_targets`
        """
        for sample in text:
            words = sample["text"].split()
            for i, curr_word in enumerate(words):
                # the "window" around the current world
                for j in range(max(0, i - self.window), min(i + self.window + 1, len(words))):
                    if i != j:
                        self.__context_words.append(self.__word2idx[curr_word])
                        self.__context_targets.append(self.__word2idx[words[j]])

    def __initialize_embedding(self) -> None:
        """
        Initialize the embedding matrix with random values
        """
        self.__embedding = np.random.uniform(-0.5 / self.dim, 0.5 / self.dim, size=(len(self.__vocab), self.dim))
    
    def __build(self, text: List[map]) -> None:
        """
        Compute and store the relevant information of the training data in the class
        """
        self.__preprocess(text)
        self.__generate_training_data(text)
        self.__initialize_embedding()

    def train(self, epochs: int=5) -> None: 
        for epoch in range(epochs):
            # learning rate decay
            learning_rate = self.lr * (1 - epoch / epochs)

            print("Training Epoch: %d" % (epoch + 1))

            for context_word, target_word in zip(self.__context_words, self.__context_targets):
                context_vector = self.__embedding[context_word]
                target_vector = self.__embedding[target_word]

                # positive sample update
                score = np.dot(target_vector, context_vector)
                exp_score = math.exp(score)
                grad_context = (exp_score / (1 + exp_score) - 1) * target_vector
                grad_target = (exp_score / (1 + exp_score) - 1) * context_vector
                self.__embedding[context_word] -= learning_rate * grad_context
                self.__embedding[target_word] -= learning_rate * grad_target

                # negative sample update
                for _ in range(self.neg):
                    negative_word = random.randint(0, len(self.__vocab) - 1)
                    if negative_word != target_word:
                        negative_vector = self.__embedding[negative_word]
                        score = np.dot(negative_vector, context_vector)
                        exp_score = math.exp(score)
                        grad_context = exp_score / (1 + exp_score) * negative_vector
                        grad_target = exp_score / (1 + exp_score) * context_vector
                        self.__embedding[context_word] -= learning_rate * grad_context
                        self.__embedding[target_word] -= learning_rate * grad_target


In [29]:
sentence = Corpus()
model = Word2Vec(
    sentences=sentence, vector_size=100, alpha=0.025, window=5, min_count=5, sample=0.001, 
    seed=1, workers=3, min_alpha=0.0001, sg=1, negative=5, ns_exponent=0.75, epochs=5, 
    sorted_vocab=1
)

### Task 2 Find similar/dissimilar word pairs

> Randomly generate 100, 1000, and 10000-word pairs from the vocabularies. For each set, print 5 closest word pairs and 5 furthest word pairs (you can use cosine-similarity to measure two words). Explain your results.

In [30]:
def generate_random_paris(samples: int):
    """
    Generate random indices without replacement, then pairs the indices to get word pairs
    """
    indices = random.sample(range(len(model.wv)), 2 * samples)
    indices1, indices2 = indices[:samples], indices[samples:]
    return [model.wv.index_to_key[i] for i in indices1], [model.wv.index_to_key[i] for i in indices2]

def find_closest_furthest(num: int=5, words1: List[str]=None, words2: List[str]=None) -> None:
    """
    Find the cloest/furthest word pairs using `model.wv.similarity`.

    Here a heap queue is used to reduce time complexity to $O(n\log k)$, where k denotes the `num`
    """
    heap = []
    for i in range(len(words1)):
        # compute the similarity and push it into the heap
        heapq.heappush(heap, (model.wv.similarity(words1[i], words2[i]), words1[i], words2[i]))
    return heapq.nlargest(num, heap), heapq.nsmallest(num, heap)[::-1]

def print_word_pairs(results: List[tuple], flag: str) -> None:
    """
    Print the result in formatted string
    """
    print("The 5 {:>8} word pairs:".format(flag))
    for result in results:
       print("Word pairs: ({:>15}, {:>15}) --> Similarity: {:>8.6f}".format(result[1], result[2], result[0]))


random.seed(408)
pairs = [100, 1000, 10000]
for pair in pairs:
    print("For {:>5} random pairs from the vocabularies:".format(pair))
    cloest, furthest = find_closest_furthest(5, *generate_random_paris(pair))
    print_word_pairs(cloest, "closest")
    print_word_pairs(furthest, "furthest")
    print("-" * 70)

For   100 random pairs from the vocabularies:
The 5  closest word pairs:
Word pairs: (      herodotus,       sorcerers) --> Similarity: 0.886167
Word pairs: (           sown,     prohibitive) --> Similarity: 0.854565
Word pairs: (        bruxing,        plumbers) --> Similarity: 0.850914
Word pairs: (stereotypically,             een) --> Similarity: 0.850355
Word pairs: (         shaded,           booby) --> Similarity: 0.811930
The 5 furthest word pairs:
Word pairs: (     violations,         yevgeny) --> Similarity: 0.238296
Word pairs: (     preferring,  neolissochilus) --> Similarity: 0.230178
Word pairs: (       superman,         outputs) --> Similarity: 0.230071
Word pairs: (     separately,       communism) --> Similarity: 0.178022
Word pairs: (        trapped,          arabia) --> Similarity: 0.151655
----------------------------------------------------------------------
For  1000 random pairs from the vocabularies:
The 5  closest word pairs:
Word pairs: (        serrano,    eng

### Task 3 Present a document as an embedding

> For each document, you have several choices to generate document embedding: 1. Use the average of embeddings of all words in each document; 2. Use the first paragraph’s words and take an average on these embeddings; 3. Use the doc2vec algorithm to present each document. Do the above for both training and testing dataset

In [36]:
####################################################################################
###        1. Use the average of embeddings of all words in each document        ###
####################################################################################
def print_document_embeddings(embeddings: Dict, display: int=3) -> None:
    """
    Print the first `display` embeddings, default value is 10
    """
    count = 0
    for title, embedding in embeddings.items():
        if count >= display:
            break
        count += 1
        print("Title: {}\nEmbedding: {}".format(title, embedding))
        print("-" * 70)

def average_all_words(data_list: List[Dict]=train_data_list, mode: str="title") -> Dict:
    doc_embeddings = {}
    if mode not in ["title", "label"]:
        raise ValueError("Please input a valid mode: ['title', 'label']")
    for line in data_list:
        line_text = utils.simple_preprocess(line["text"]) # preprocess the text as in class `Corpus`
        # doc title/label and doc embedding computed by averaging all words embeddings
        doc_info = line[mode]
        valid_word_embeddings = [model.wv[word] for word in line_text if word in model.wv]
        doc_embedding = sum(valid_word_embeddings) / len(valid_word_embeddings)
        # store the information in a dictionary
        doc_embeddings[doc_info] = doc_embedding

    return doc_embeddings

print("For the training dataset:")
print_document_embeddings(average_all_words())
print("-" * 70)
print("For the  testing dataset:")
print_document_embeddings(average_all_words(test_data_list))


For the training dataset:
Title: Citizen_Kane
Embedding: [-2.50620186e-01 -1.24460168e-01  1.07301518e-01  1.06681667e-01
 -2.21395597e-01 -3.52198243e-01  2.12455407e-01  3.53895932e-01
 -2.15869695e-01 -3.49741787e-01 -5.18604964e-02 -3.69868010e-01
  3.14556248e-02  1.32968366e-01  3.22128274e-02 -1.76448002e-01
  9.24646705e-02  8.66957288e-03 -8.88008401e-02 -3.57388258e-01
  3.24919894e-02  2.21586511e-01  1.33043960e-01  1.34189054e-01
  3.74840684e-02  1.87552407e-01 -2.69486457e-01  1.18333451e-01
 -1.14250392e-01 -1.12703137e-01 -2.77006570e-02  1.69034123e-01
 -9.03501064e-02 -4.79165353e-02  1.85195042e-03  2.02847227e-01
 -8.63823816e-02 -1.64019570e-01 -4.84447666e-02 -2.38638595e-01
 -2.54364070e-02 -2.61652976e-01 -3.04982334e-01 -1.23209968e-01
  3.49789172e-01 -1.77557100e-04 -1.61893055e-01 -2.29572989e-02
  7.82075897e-02 -7.45770931e-02  8.86614993e-02  2.33614668e-02
 -1.36879414e-01 -1.10366493e-01 -7.30106160e-02  1.19938485e-01
 -1.05985805e-01  8.51468965e-02 

In [32]:
####################################################################################
###  2. Use the first paragraph’s words and take an average on these embeddings  ###
####################################################################################
def find_first_paragraph(text: str):
    return text.split("\n")[0]

def average_first_para_words(data_list: List[Dict]=train_data_list, mode: str="title") -> Dict:
    doc_embeddings = {}
    if mode not in ["title", "label"]:
        raise ValueError("Please input a valid mode: ['title', 'label']")
    for line in data_list:
        doc_info = line[mode]
        valid_word_embeddings = [model.wv[word] for word in utils.simple_preprocess(find_first_paragraph(line["text"])) if word in model.wv]
        # since the number of words in the first paragraph is so small, it may occur `ZeroDivisonError` in the computation, here I use a 
        # try-except flow to handle this exception
        try:
            doc_embedding = sum(valid_word_embeddings) / len(valid_word_embeddings)
        except:
            doc_embedding = [0 for _ in range(len(valid_word_embeddings))]
        doc_embeddings[doc_info] = doc_embedding


    return doc_embeddings

average_first_para_words_embedding = average_first_para_words()
print_document_embeddings(average_first_para_words_embedding)

Citizen_Kane:
[-2.55847305e-01 -9.10092816e-02  2.11859465e-01  9.32272747e-02
 -1.63740218e-01 -4.45404261e-01  2.55138785e-01  3.16293001e-01
 -2.67182022e-01 -3.24095517e-01 -1.37831882e-01 -4.00891244e-01
  2.40017889e-05  4.15285788e-02  2.55168062e-02 -2.56034017e-01
  1.12494066e-01 -8.41944069e-02 -8.90707895e-02 -3.30588758e-01
 -4.00519930e-02  3.16697747e-01  1.17731757e-01  4.68406305e-02
  5.03130369e-02  1.96792975e-01 -2.86228806e-01  1.43448085e-01
 -6.05412014e-02 -8.46401453e-02  2.22788304e-02  2.00624779e-01
 -1.87066823e-01 -3.05731241e-02  1.93206612e-02  2.33878955e-01
 -1.47629499e-01 -1.38299137e-01 -5.80723062e-02 -2.44014874e-01
  1.52438255e-02 -3.16461951e-01 -3.14271510e-01 -9.72499847e-02
  3.41878742e-01  3.40494956e-03 -1.16632216e-01  3.97241004e-02
 -1.17706200e-02 -9.07134637e-02  1.66977808e-01  8.40164945e-02
 -1.14028901e-01 -7.27838129e-02 -6.90624714e-02  3.25847454e-02
 -9.35799405e-02  9.07470286e-02  5.53231388e-02  1.39155984e-01
  1.4528670

In [33]:
####################################################################################
###             3. Use the doc2vec algorithm to present each document            ###
####################################################################################
def build_doc2vec() -> Doc2Vec:
    """
    Build the `Doc2Vec` model based on the `train_data_list`
    """
    tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate([line["text"] for line in train_data_list])]
    model = Doc2Vec(vector_size=100, window=5, min_count=5, epochs=10)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

doc2vec_model = build_doc2vec()

def doc2vec_model_embeddings(data_list: List[Dict]=train_data_list, mode: str="title") -> tuple:
    embeddings = {}
    if mode not in ["title", "label"]:
        ValueError("Please input a valid mode: ['title', 'label']")
    for line in data_list:
        embeddings[line[mode]] = doc2vec_model.infer_vector(line["text"].split())

    return embeddings

doc2vec_model_embedding = doc2vec_model_embeddings()
print_document_embeddings(doc2vec_model_embedding)

Citizen_Kane:
[ 0.5673132   2.5735762  -2.4015522   1.4646828   2.4414494   0.23134758
 -0.7582788   1.4945705  -1.2989575  -0.5255742   3.6375487   1.2254568
  0.31390887 -0.2291875  -3.7744157  -0.46353722 -2.3252726   1.150479
 -4.9491506   1.2770969  -3.5225692  -1.2958678   0.58023304  1.9822471
 -3.9455993  -3.185178   -3.0894163  -0.43998092  4.9467807  -2.748377
 -2.1442611  -0.11394686  1.90851    -0.13847026 -0.10003136 -0.466067
  0.05659573 -4.338431   -1.7576983   2.7716398  -1.7794051  -1.3922782
  2.7097208  -5.3522778   1.5797373  -5.0972633   3.58482     0.3739586
  1.1834635   2.3040378   2.143242    2.9439483   1.2628907  -3.2546782
  1.4437549   1.6612926  -0.9021918   0.99941665 -4.7778273   0.36283186
 -3.8838646   3.8384051   1.9242661  -2.3999853  -0.79136986  0.56406
  2.6099274  -2.3764508   0.6013121  -2.1730378  -3.0501795   5.2701416
 -3.2049825  -4.548285   -0.66558415  2.3368027   1.7597922  -2.7117915
 -0.04448551  5.824465   -3.473989    3.1053188  -2.1

### Task 4 Build classifier to test docs
> Build softmax regression model to classifier testing documents based on these training doc embeddings. Does it getting better than Naive Bayes'? (You have 3 models.)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

class softmaxModel:
    def __init__(self, data: List[Dict], method: Callable) -> None:
        self.X = None
        self.y = None
        self.__model = None
        self.__ytest = None
        self.__ypred = None
        self.__method = method               # the three different models in task 3
        self.__train(method(data, "label"))  # use `method(data)` to get the embedding

    def __train(self, embedding: Dict):
        """
        Train a softmax classifier based on the embedding in train data
        """
        X, y = list(embedding.values()), list(embedding.keys())
        self.X, self.y = X, y
        self.__model = LogisticRegression(multi_class="multinomial", solver="lbfgs").fit(self.X, self.y)


    def predict(self, test_data: List[Dict]) -> List:
        """
        Return the pred value of y based on the trained model and given test data
        """
        test_embedding = self.__method(test_data, "label")
        self.__ytest = list(test_embedding.keys())
        self.__ypred = self.__model.predict(list(test_embedding.values()))
        return self.__ypred

    def accuracy(self) -> float:
        return accuracy_score(self.__ytest, self.__ypred)
    
methods = {
    "Use the average of embeddings of all words in each document": average_all_words, 
    "Use the first paragraph’s words and take an average on these embeddings": average_first_para_words, 
    "Use the doc2vec algorithm to present each document": doc2vec_model_embeddings
}

for text, method in methods.items():
    softmax_model = softmaxModel(train_data_list, method)
    softmax_model.predict(test_data_list)
    print("Model: {:>71}, accuracy: {:>.6f}".format(text, softmax_model.accuracy()))
    del softmax_model


Model:             Use the average of embeddings of all words in each document, accuracy: 0.000000


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

### Task 5 Use t-SNE to project doc vectors

> Use t-SNE to project training document embeddings into 2d and plot them out for each of the above choices. Each point should have a specific color (represent a particular cluster). You may need to try different parameters of t-SNE. One can find more details about t-SNE in this [excellent article](https://distill.pub/2016/misread-tsne/).

In [ ]:
# Your code


